In [1]:
!pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.2 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 2.3 MB 39.6 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=e28fc50d660c0f7843e8590b63cbbfd65e8b8c106e128f8bfb0d5572325800df
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [2]:
import gdown
## google drive에서 압축된 dataset download
url = 'https://drive.google.com/uc?id=1xtowuN7dZRAzPMpLzpb2lmwInq6zkrBD'
fname = 'data.zip'
gdown.download(url, fname, quiet=False)
!unzip data.zip -d data # test를 위한 data.zip 파일 가져오기.

Downloading...
From: https://drive.google.com/uc?id=1xtowuN7dZRAzPMpLzpb2lmwInq6zkrBD
To: /content/data.zip
100%|██████████| 57.1M/57.1M [00:00<00:00, 219MB/s]


Archive:  data.zip
  inflating: data/cjh/1.jpg          
  inflating: data/cjh/10.jpg         
  inflating: data/cjh/11.jpg         
  inflating: data/cjh/12.jpg         
  inflating: data/cjh/13.jpg         
  inflating: data/cjh/14.jpg         
  inflating: data/cjh/15.jpg         
  inflating: data/cjh/16.jpg         
  inflating: data/cjh/17.jpg         
  inflating: data/cjh/2.jpg          
  inflating: data/cjh/3.jpg          
  inflating: data/cjh/4.jpg          
  inflating: data/cjh/5.jpg          
  inflating: data/cjh/6.jpg          
  inflating: data/cjh/7.jpg          
  inflating: data/cjh/8.jpg          
  inflating: data/cjh/9.jpg          
  inflating: data/ksr/1.jpg          
 extracting: data/ksr/2.jpg          
  inflating: data/ksr/3.jpg          
  inflating: data/ksr/4.jpg          
  inflating: data/many.jpg           
  inflating: data/one_cjh1.jpg       
  inflating: data/one_cjh2.jpg       
  inflating: data/one_cjh3.jpg       
  inflating: data/one_cjh4.jpg 

In [3]:
# !unzip data.zip -d data

# img -> pickle로 저장

## data에 저장된 파일을 pkl 형식으로 저장

- pkl 내부 (딕셔너리 형식)
    - {"id1":data(list), "id2":data(list)}

```
data 
    id1 (파일 이름이 피해자 초성)
        1.jpg (형식 상관 없음 일단 jpg로 설정. 원하면 다른 형식도 추가)
        2.jpg
        ...
    id2
        1.jpg
        2.jpg
        ...
    ...
```

In [4]:
# makeVdata.py

from deepface import DeepFace
from glob import glob
from tqdm import tqdm
import pickle, os


def make_pickle(emb_dic, datapath):
    '''
    import os, pickle
    경로 commons/
    '''
    pickle_name = 'dataset.pkl'

    try : # 기존 pickle이 존재
        f = open(datapath+"/"+pickle_name, 'rb')
        rb = pickle.load(f)
        f.close()
        for d_name in rb.keys(): 
            if d_name in emb_dic.keys(): # 기존 이름 있으면 데이터 추가
                rb[d_name].extend(emb_dic[d_name])
            else :
                rb[d_name] = emb_dic[d_name]
        f = open(datapath+"/"+pickle_name, 'wb')
        data = pickle.dump(rb, f)
        f.close
    except:
        f = open(datapath+"/"+pickle_name, 'wb')
        data = pickle.dump(emb_dic, f)
        f.close
    
    with open(datapath+"/"+pickle_name, 'rb') as f:
        rb = pickle.load(f)
        f.close
    return rb

def rm_dir(paths): # 파일 삭제.
    for path in paths:
        path = "/".join(path.split("/")[:-1])+"/"
        try:
            os.rmdir(path)
        except :
            continue

#./data/{name}/{num}.jpg
def embeded_file(datapath): 
    img_path_list = []
    img_path_list.extend(glob(os.path.join(datapath, '*/*.jpg')))

    emb_dic = {}
    for img_path in tqdm(img_path_list):
        img_path_split = img_path.split('/')
        d_name = img_path_split[2] # 디렉토리 이름
        try :
            embedding = DeepFace.represent(img_path=img_path, model_name='Facenet512', detector_backend="mtcnn") # img_path 변경
        except:
            pass
        os.remove(img_path)
        if d_name not in emb_dic.keys() :
            emb_dic[d_name] = [embedding]
        else :
            emb_dic[d_name].append(embedding)
    rm_dir(img_path_list)
    return emb_dic, datapath

emb_dic, datapath = embeded_file('./data')

data = make_pickle(emb_dic, datapath)


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


  0%|          | 0/21 [00:00<?, ?it/s]

facenet512_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5

  0%|          | 0.00/95.0M [00:00<?, ?B/s]
  1%|          | 1.05M/95.0M [00:00<00:16, 5.72MB/s]
  2%|▏         | 2.10M/95.0M [00:00<00:17, 5.42MB/s]
  4%|▍         | 3.67M/95.0M [00:00<00:13, 6.52MB/s]
  5%|▍         | 4.72M/95.0M [00:00<00:16, 5.34MB/s]
  6%|▌         | 5.77M/95.0M [00:01<00:16, 5.29MB/s]
  7%|▋         | 6.82M/95.0M [00:01<00:16, 5.28MB/s]
  9%|▉         | 8.39M/95.0M [00:01<00:18, 4.62MB/s]
  9%|▉         | 8.91M/95.0M [00:01<00:21, 4.02MB/s]
 10%|▉         | 9.44M/95.0M [00:02<00:23, 3.67MB/s]
 11%|█         | 10.5M/95.0M [00:02<00:20, 4.08MB/s]
 12%|█▏        | 11.5M/95.0M [00:02<00:18, 4.41MB/s]
 13%|█▎        | 12.6M/95.0M [00:02<00:17, 4.64MB/s]
 14%|█▍        | 13.1M/95.0M [00:02<00:20, 3.96MB/s]
 14%|█▍        | 13.6M/95.0M [00:03<00:22, 3.58MB/s]
 15%|█▌        | 14.7M/95.0M [00:03<00:19, 4.06MB/s]


## 테스트

In [5]:
f = open('./data/dataset.pkl', "rb")
data = pickle.load(f)

In [6]:
data

{'cjh': [[0.5599257946014404,
   0.7184942960739136,
   -0.8235679864883423,
   0.8799230456352234,
   0.7705445289611816,
   -0.06280698627233505,
   -1.79381263256073,
   0.92999666929245,
   0.8142094016075134,
   0.5750491619110107,
   1.4572023153305054,
   1.8763655424118042,
   -0.03190534934401512,
   -0.06221042200922966,
   0.3945270776748657,
   1.1013811826705933,
   2.073096513748169,
   1.7886831760406494,
   -0.27299413084983826,
   0.4259394705295563,
   0.2730974853038788,
   -0.6100576519966125,
   0.6366192102432251,
   -1.8413721323013306,
   -0.5499112010002136,
   -0.780146598815918,
   1.256606936454773,
   0.5596567392349243,
   0.9365710616111755,
   0.813346803188324,
   1.2457627058029175,
   -0.8670558333396912,
   0.5403342843055725,
   1.7259587049484253,
   -0.9919410347938538,
   -1.3565001487731934,
   -1.3185783624649048,
   -1.4373869895935059,
   -1.0709987878799438,
   2.2986721992492676,
   -0.7110074162483215,
   -1.9122408628463745,
   0.01600600

# 영상을 받아 pickle과 대조

In [7]:
import cv2
import pickle
import time

from deepface import DeepFace
from deepface.commons import distance
from deepface.detectors import FaceDetector

from google.colab.patches import cv2_imshow

In [ ]:
image_file = "./data/many.jpg" # 변경
encoding_file = './data/dataset.pkl' # 고정
model_name = 'Facenet512' # 고정
detector_backend = "mtcnn" # 고정
id = 'cjh' # 변경

# box 데이터와 embeding 데이터 분리
def splitData(obj):
    encodings = []
    boxes = []
    print(obj)
    for o in obj:
        try :
            encodings.append(DeepFace.represent(img_path=o[0], model_name=model_name, detector_backend=detector_backend)) # embeded 한 데이터를 저장한다.
            startX, StartY, endX, endY = o[1][0],o[1][1],o[1][0]+o[1][2],o[1][1]+o[1][3]    # 각 좌표를 저장
            boxes.append([startX, StartY, endX, endY]) 
        except:
            print(1)
    return encodings, boxes


# 확인 후 비교
def detectAndDisplay(image, id):
    start_time = time.time()
    face_detector = FaceDetector.build_model(detector_backend)
    obj = FaceDetector.detect_faces(face_detector=face_detector,
                                    detector_backend=detector_backend, img=image) # 사진과 얼굴 좌표를 가져온다 [[이미지], [좌표]]
    encodings, boxes = splitData(obj)
    threshold = distance.findThreshold(model_name, 'cosine') # 정답 0.3
    dist_cnt = [0 for i in range(len(encodings))]

    # 
    for i, (encoding,(startX, startY, endX, endY)) in enumerate(zip(encodings, boxes)): 
        for da in data[id]:
            dist = distance.findCosineDistance(encoding, da) # 거리값을 하나씩 계산 cosine 로 사용
            if dist <= threshold: # threshold보다 작으면 cnt
                dist_cnt[i] += 1

    max_cnt = max(dist_cnt)

    for i, (encoding,(startX, startY, endX, endY)) in enumerate(zip(encodings, boxes)):
        name = "UnKnown"    # 정답 기준을 못넘었을 경우
        color = (0, 255, 0) # 기본 green 색상

        if dist_cnt[i] == max_cnt:
            cnt_avg=max_cnt/len(encodings)
            name = id+" "+str(round(cnt_avg,3) * 100)+"%"
            color = (255, 0, 0)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2) # 얼굴 사각형 그려주기
        y = startY - 15 if startY - 15 > 15 else startY + 15            # 화면을 벗어날수 있어 예외처리 
        cv2.putText(image, name, (startX, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, color, 2) # 글자 입력 처리


    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    cv2_imshow(image)               # cv2.imshow("image", image)

data = pickle.loads(open(encoding_file, "rb").read())

image = cv2.imread(image_file)
detectAndDisplay(image, id)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 얼굴을 잘 인식하고 embeded값을 잘 가져오는지 테스트용

- 8 = 손가락을 인식을 했다.

In [ ]:
from deepface.detectors import FaceDetector
image_file = "./data/many.jpg"
emb1 = pickle.loads(open('./data/dataset.pkl',"rb").read())
image = cv2.imread(image_file)
face_detector = FaceDetector.build_model('mtcnn')
obj = FaceDetector.detect_faces(face_detector=face_detector, detector_backend="mtcnn",img=image)
emb2 = []
for i,o in enumerate(obj):
    try :
        emb2.append((DeepFace.represent(img_path=o[0], model_name='Facenet512', detector_backend="mtcnn"),i))
        print(i, True)
    except :
        print(i, False)

## 사진의 거리값이 threshold보다 작은지 테스트용

In [ ]:
for emb,j in emb2:
    print(f'-------{j}-----')
    for i in range(len(emb1['cjh'])):
        a = distance.findCosineDistance(emb, emb1['cjh'][i])
        threshold = distance.findThreshold('Facenet512', 'cosine')
        if a <= threshold:
            print(True, j,"번째 사진", a, threshold)
        else :
            print(False, j, "번째 사진", a, threshold)
    


-------0-----
False 0 번째 사진 0.4256941714384931 0.3
False 0 번째 사진 0.422176780463818 0.3
False 0 번째 사진 0.733253976688542 0.3
False 0 번째 사진 0.40738618848748354 0.3
False 0 번째 사진 0.4211200526642914 0.3
False 0 번째 사진 0.46829152594392576 0.3
False 0 번째 사진 0.5893987142489409 0.3
False 0 번째 사진 0.6347206446435963 0.3
False 0 번째 사진 0.5397330700939722 0.3
False 0 번째 사진 0.5270488745047044 0.3
False 0 번째 사진 0.7051328291245984 0.3
False 0 번째 사진 0.44498708875776816 0.3
False 0 번째 사진 0.6320158752418444 0.3
False 0 번째 사진 0.4284887564785561 0.3
False 0 번째 사진 0.3949349624904638 0.3
False 0 번째 사진 0.46566032046350814 0.3
False 0 번째 사진 0.4193176846330857 0.3
-------1-----
True 1 번째 사진 0.2676213101450329 0.3
False 1 번째 사진 0.466747111885076 0.3
False 1 번째 사진 0.6608477217221558 0.3
False 1 번째 사진 0.5889075387943212 0.3
False 1 번째 사진 0.4384966979879231 0.3
False 1 번째 사진 0.5306939766823717 0.3
False 1 번째 사진 0.4548541297360479 0.3
False 1 번째 사진 0.6544651358705451 0.3
False 1 번째 사진 0.616469924724112 0.3
False 1 번째 

## 얼굴의 순서를 눈으로 확인하기 위해 구현
- 내 얼굴은 4번
- 7번 옆 모습도 잘 인식한 것을 확인
- 8번으로 인해 오류가 생길수 있어 try ~ except 구문 생성

In [ ]:
for i,o in enumerate(obj):
    cv2.rectangle(image, (o[1][0], o[1][1]), (o[1][0]+o[1][2], o[1][1]+o[1][3]), (0, 255,0), 2)
    y = o[1][1] - 15 if o[1][1] - 15 > 15 else o[1][1] + 15
    cv2.putText(image, f'{i}', (o[1][0], y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0,255,0), 2)
cv2_imshow(image)

Output hidden; open in https://colab.research.google.com to view.